In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import *
import numpy as np
import sys, os
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
from _global.bbx import *
from _global.recognition import *
sys.path.extend([RESEARCH])
from object_detection.utils import config_util
from object_detection.builders import model_builder
from IPython import get_ipython
import cv2 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
import urllib.request
ipython = get_ipython()

In [ ]:
def prepare_and_get_dataset(img_url_paths, label_name, label_num):
    t = os.path.join(INSTALLATION_PATH, "TEMP_IMGS")
    if os.path.exists(t):
        os.rmdir(t)
    os.mkdir(t)

    for i in range(len(img_url_paths)):
         urllib.request.urlretrieve(img_url_paths[i], os.path.join(t, "temp{}.jpg".format(i)))
         
    temp_tfrecord_name = os.path.join(t, "temp.tfrecord")
    parse_images(temp_tfrecord_name, t, label_name, label_num)

    dataset = tf.data.TFRecordDataset([temp_tfrecord_name])
    dataset = dataset.map(parse_fn)
    dataset = dataset.batch(FACE_RECOG_BATCH_SIZE)

    os.rmdir(t)
    return dataset

In [ ]:
def new_face_added(model, face_name, img_url_paths, override_existing=True):
    curr_classes = model.layers[:].pop().output_shape[-1]
    last_dense_before_output_neurons = model.layers[:][len(model.layers)-1]
    
    prev_output_layer = model.layers.pop()
    prev_weights = prev_output_layer.get_weights()[0]
    bias = prev_weights[1]
    sum = 1
    for shape in prev_weights.shape:
        sum *= shape
    prev_weights = np.reshape(prev_weights, (sum))
    prev_weights = np.array(prev_weights.tolist().append(bias))
    curr_classes += 1
    for _ in range(last_dense_before_output_neurons):
        prev_weights = np.insert(prev_weights, 0, 0.0, 0)
    
    new_init_weights = tf.constant_initializer(prev_weights)
    new_output_layer = layers.Dense(curr_classes, activation='softmax', kernel_initializer=new_init_weights) 
    model.add(new_output_layer)

    dataset = prepare_and_get_dataset(img_url_paths, face_name, curr_classes)
    model.fit(dataset, epochs=3)

    extend_label_map(curr_classes +1, face_name)

    if override_existing:
        models.save_model(model, FACES_RECOG)

    return model